# Esercizio 46 - Assignment problem

The problem instance has a number of agents and a number of tasks. Any agent can be assigned to perform any task, incurring some cost that may vary depending on the agent-task assignment. It is required to perform all tasks by assigning exactly one agent to each task and at most one task to each agent, in such a way that the total cost of the assignment is minimized.

<img src="https://www.tommasoadamo.it/images/lez19/1.svg" style="width: 500px; margin:auto;"/>

## Solution by linear programming

\begin{align}
\min\quad & \sum\limits_{(i,j) \in A \times T} w_{ij} x_{ij}&\\
s.t.\quad & \sum\limits_{j \in T} x_{ij} \leq 1 & i \in A\\
\quad & \sum\limits_{i \in A} x_{ij} = 1 & j \in T\\
\quad & x_{ij} \geq 0 & i \in A, j \in T
\end{align}

There is one variable for each pair of agent and task. Each agent is assigned to at most 1 task. Each task is assigned to exactly one agent.

While this formulation allows also fractional variable values, in this special case, the LP always has an optimal solution where the variables take integer values. This is because the constraint matrix of the fractional LP is totally unimodular – it satisfies the four conditions of Hoffman and Gale.
 

Esempio: file _costi.csv_<br>
Minimum cost assignment: 235<br>
Agent 0 assigned to task 3 with cost 70<br>
Agent 1 assigned to task 2 with cost 55<br>
Agent 3 assigned to task 0 with cost 45<br>
Agent 5 assigned to task 1 with cost 65

<img src="https://www.tommasoadamo.it/images/lez19/2.svg" style="width: 500px; margin:auto;"/>

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/costi.csv')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import csv
import docplex.mp.model as cplex

def leggi_grafo(nome):
    with open(nome, 'r') as file:
        reader = csv.reader(file, quoting=2)
        return list(reader)
# leggere il file CSV e preparare i dati

w = leggi_grafo('costi.csv')
A = list(range(len(w)))
T = list(range(len(w[0])))

# formulare il modello

with cplex.Model('assegnamento') as mdl:
    x = mdl.continuous_var_matrix(A, T, name='x')
    
    mdl.minimize(sum(w[i][j] * x[i,j] for i in A for j in T))
    
    for i in A:
        mdl.add_constraint(sum(x[i,j] for j in T) <= 1, 'assegna_agente_'+str(i))
    
    for j in T:
        mdl.add_constraint(sum(x[i,j] for i in A) == 1, 'assegna_task_'+str(j))
    
    mdl.print_information()
    
    s = mdl.solve()
    
    if s == None:
        print('No solution.')
    else:
        #print(s)
        print('Costo totale dell\'assegnamento: '+str(s.objective_value))
        for i in A:
            for j in T:
                if x[i,j].solution_value > 0:
                    print('Agente A{} assegnato al task T{} con costo {}'.format(i,j,w[i][j]))